In [ ]:
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

::::{margin}
:::{card} Rotated dispersion integral path
TR-981
^^^
As a follow-up to [TR-980](./980.ipynb), we rotate the cut of phase space factor **and** the path of the dispersion integral.
+++
🚧&nbsp;[compwa.github.io#204](https://github.com/ComPWA/compwa.github.io/pull/204)
:::
::::

# Rotated dispersion integral path

In [ ]:
%pip install -q numpy==1.24.4 scipy==1.10.1

In [ ]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import quad_vec as integrate

warnings.filterwarnings("ignore")

:::{admonition} General dispersion integral
$$
f(z) = \frac{1}{2\pi i} \int_{-\infty}^\infty \frac{f(z')}{z'- z} \mathop{}\!\mathrm{d}z'
$$
:::

## Branch cut with rotation 
In the following a representation of the integral is derived where it is possible to rotate the branch cut by an arbitrary angle $\phi$. The necessary information is taken from  [O. Deineka, dissertation (p.21-22)](https://openscience.ub.uni-mainz.de/handle/20.500.12030/8998).
## Derivation from line integral 
The $f(z)$ is a complex valued function using Cauchy’s integral formula the line integral around an arbitrary point reads: 
$$
\frac{1}{2\pi i} \oint_{c}\frac{f(z')}{z'- z} \mathop{}\!\mathrm{d}z'
$$

the contour $c$ describes a closed circle around the point.

:::{admonition} [PDG2023, §Resonances, Eq. (50.43)](https://pdg.lbl.gov/2023/reviews/rpp2023-rev-resonances.pdf#page=15)

$$
\begin{eqnarray}
\Sigma(s + 0i) &=&
\frac{s-s_\mathrm{thr}}{\pi}
\int_{s_\mathrm{thr}}^\infty \frac{
    \rho(s')
}{
    (s' - s_\mathrm{thr})(s'-s-i0)
}
\mathop{}\!\mathrm{d}s'
\qquad \text{original from the PDG}
\\
&=&
\frac{s-s_\mathrm{thr}}{2\pi}
\left[
\int_{s_\mathrm{thr}}^\infty \frac{
    \rho(s'+0i)
}{
    (s' - s_\mathrm{thr})(s'+i0-s)
}
\mathop{}\!\mathrm{d}s'
-
\int_{s_\mathrm{thr}}^\infty \frac{
    \rho(s'-0i)
}{
    (s' - s_\mathrm{thr})(s'-i0-s)
}
\mathop{}\!\mathrm{d}s'
\right]
\end{eqnarray}
$$

:::

$$
s' = s_\mathrm{thr} + x e^{i\phi} + \epsilon i, \quad
\mathrm{d}s' =  e^{-i\phi} \mathrm{d}x, \quad
x \in \left[0, \infty\right)
$$

$$
\frac{s-s_{\mathrm{thr}}}{2\pi}
\left[
\int^\infty_0 \frac{
    \rho(s_\mathrm{thr} + x e^{i\phi} + 0i)
}{
    \left(x e^{i\phi} + 0i\right)
    \left(s_\mathrm{thr} + x e^{i\phi} + 0i - s\right)
}
e^{-i\phi}
\mathop{}\!\mathrm{d}x
-
\int^\infty_0 \frac{
    \rho(s_\mathrm{thr} + x e^{i\phi} - 0i)
}{
    \left(x e^{i\phi} - 0i\right)
    \left(s_\mathrm{thr} + x e^{i\phi} - 0i - s\right)
}
e^{-i\phi}
\mathop{}\!\mathrm{d}x
\right]
$$

In [ ]:
ϵi = 1e-10 * 1j


def rho_cm(s, m1, m2, phi: float = 0) -> np.ndarray:
    return -1j * dispersion_integral(s, m1, m2, phi)


def dispersion_integral(s, m1, m2, phi: float = 0) -> np.ndarray:
    s_thr = (m1 + m2) ** 2
    exp = np.exp(phi * 1j)
    func1 = (
        lambda x: rho(s_thr + x * exp + ϵi, m1, m2, phi)
        / ((x * exp + ϵi) * (s_thr + x * exp + ϵi - s))
        * np.exp(-phi * 1j)
    )
    func2 = (
        lambda x: rho(s_thr + x * exp - ϵi, m1, m2, phi)
        / ((x * exp - ϵi) * (s_thr + x * exp - ϵi - s))
        * np.exp(-phi * 1j)
    )
    return (
        (s - s_thr)
        / (2 * np.pi)
        * (integrate(func1, 0, np.inf)[0] - integrate(func2, 0, np.inf)[0])
    )


def rho(s, m1, m2, phi: float = 0) -> np.ndarray:
    return rotated_sqrt((s - (m1 + m2) ** 2) * (s - (m1 - m2) ** 2), phi) / s


def rotated_sqrt(z, phi: float = 0) -> np.ndarray:
    return np.exp(-0.5j * phi) * np.sqrt(z * np.exp(phi * 1j))

In [ ]:
%config InlineBackend.figure_formats = ['svg']
resolution = 100
x = np.linspace(0, 3, num=resolution)
m1 = 0.1
m2 = 0.7
phi = -np.pi / 6

y1 = rho(x, m1, m2, phi)
y2 = rho_cm(x, m1, m2, phi)
assert not any(np.isnan(y2)), "Contains NaN values!"

fig, ax = plt.subplots()
ax.axhline(0, c="black", lw=0.4)
ax.axvline(0, c="black", lw=0.4)
ax.plot(x, y1.real, label=R"$\rho$", lw=4, alpha=0.5)
ax.plot(x, y2.real, label=R"$\rho^\mathrm{CM}$")
ax.axvline((m1 - m2) ** 2, c="red", ls="dotted", label="$(m_1-m_2)^2$")
ax.axvline((m1 + m2) ** 2, c="black", ls="dotted", label="$(m_1+m_2)^2$")
ax.set_ylim(-0.3, +1.6)
ax.set_xlabel("$s$")
ax.set_title(Rf"$\phi={phi / np.pi:.4g}\pi$")
ax.legend()
fig.tight_layout()
plt.show()

In [ ]:
y = np.linspace(-1, +1, num=resolution)
X, Y = np.meshgrid(x, y)
Z = X + Y * 1j
T = rho_cm(Z, m1, m2, phi)

%config InlineBackend.figure_formats = ['png']
fig, ax = plt.subplots()
ax.set_xlabel(R"$\mathrm{Re}\,s$")
ax.set_ylabel(R"$\mathrm{Im}\,s$")
ax.set_title(Rf"$\phi={phi / np.pi:.4g}\pi$")
mesh = ax.pcolormesh(X, Y, T.real, cmap=plt.cm.coolwarm)
plt.colorbar(mesh, ax=ax, pad=0.02)
fig.tight_layout()
plt.show()